<a href="https://colab.research.google.com/github/kghilothia06/Time-Series-Forecasting/blob/main/02_vector_output_model_9_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  univariate multi-step vector-output stacked lstm example

Import Libraries

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd


function to split a univariate sequence into samples

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the sequence
    if out_end_ix > len(sequence):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

Function to define model

In [3]:
def build_and_train_model(X_train, y_train, X_test, y_test, n_steps_in, n_steps_out, n_features):
  # build model
  model = Sequential()
  model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in,
  n_features)))
  model.add(LSTM(100, activation='relu'))
  model.add(Dense(n_steps_out))
  model.compile(optimizer='adam', loss='mse')
  print('Model Built!')
  # fit model
  model.fit(X_train, y_train, epochs=50, verbose=0)
  print('Model Trained!')
  # make prediction
  print('Now making predictions!')
  yhat = model.predict(X_test, verbose=0)
  print('Predicted : ' + str(yhat))
  print('Actual : ' + str(y_test))

Function for data preparation

In [4]:
def Data_prep(df):
  # choose a number of time steps
  n_steps_in, n_steps_out = 3, 2

  #Convert 'Sales' column to a list
  raw_seq=[]

  for i in df['Sales']:
    raw_seq.append(i)

  # split into samples
  X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
  
  # taking all the data except last row
  X_train = X[:len(X)-1]
  y_train = y[:len(y)-1]
  
  # test sample would be data corresponding to last row
  X_test = X[len(X)-1]
  y_test = y[len(y)-1]


  # reshape from [samples, timesteps] into [samples, timesteps, features]
  n_features = 1
  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
  X_test = X_test.reshape(((1, n_steps_in, n_features)))
  print('Data Preparation DONE!')

  #build and train model
  build_and_train_model(X_train, y_train, X_test, y_test, n_steps_in, n_steps_out, n_features)
  

Load quarterly and monthly datasets and train the LSTM model

In [5]:
df1 = pd.read_excel('/content/Sales-quarterly.xlsx',header=0,index_col=0)

In [6]:
Data_prep(df1)

Data Preparation DONE!
Model Built!
Model Trained!
Now making predictions!
Predicted : [[46138.324 41864.164]]
Actual : [204660.251 611909.638]


In [7]:
df2 = pd.read_excel('/content/Sales-monthly.xlsx',header=0,index_col=0)

In [8]:
Data_prep(df2)

Data Preparation DONE!
Model Built!
Model Trained!
Now making predictions!
Predicted : [[ 78107.81 111505.96]]
Actual : [ 54648.113 513890.482]
